# Imports

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/TRAINING

In [2]:
# %%capture
# !pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, util
from tqdm import tqdm
import random
from sentence_transformers import evaluation
import json
import pandas as pd
from collections import Counter
import os
from sklearn.model_selection import train_test_split
from helpers import load_data_pairs, flatten_list,load_json
import numpy as np
from datetime import datetime
from sklearn.model_selection import KFold
from transformers import set_seed
set_seed(42)

c:\Users\lw\miniconda3\envs\masterarbeit_lwo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def create_trainig_samples(pos_dev_samples,neg_pairs):
  dev_set_total =[]
  anchors = set([x[0] for x in pos_dev_samples])
  neg_dev_samples = [x for x in neg_pairs if x[0] in anchors]
  print("Creating Dev Set")
  for anchor in tqdm(anchors):
    pos_pairs_filtered = [x[1] for x in pos_dev_samples if x[0]==anchor]
    neg_pairs_filtered = [x[1] for x in neg_dev_samples if x[0]==anchor]
    dev_set_total.append({"query":anchor,"positive":pos_pairs_filtered,"negative":neg_pairs_filtered})
  return dev_set_total

## Pretrained Model

# Load data

In [6]:
data_dict = load_data_pairs()
pos_pairs = flatten_list([data_dict[x] for x in data_dict if "pos" in x])
neg_pairs = flatten_list([data_dict[x] for x in data_dict if "neg" in x])

100%|██████████| 6/6 [00:02<00:00,  2.67it/s]


In [14]:
testads = pd.DataFrame(load_json("../00_data/EURES/eures_testads_final_short.json"))

# Pre-Training

##  TSDAE

In [ ]:
pretraining_list = list(set(flatten_list(pos_pairs)))
len(pretraining_list)

In [ ]:
# jobgbert model
#model = SentenceTransformer("agne/jobgbert")
#gbert model 
model = SentenceTransformer("deepset/gbert-base")

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

pretraining_dataset = datasets.DenoisingAutoEncoderDataset(pretraining_list)
train_dataloader = DataLoader(pretraining_dataset, batch_size=16, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs = 5,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)
model.save("../00_data/SBERT_Models/models/gbert_TSDAE_epochs5")